<a href="https://colab.research.google.com/github/imaman2001/Machine-Learning-Project-1/blob/main/weather_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [2]:
API_KEY = "24e66b0a6976f43a7ff033dbc1014397"
Base_URL = "https://api.openweathermap.org/data/2.5/"

Fetch current weather **data**

In [18]:
def get_current_weather(city):
  url = f"{Base_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),6
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir' : data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'Wind_Gust_Speed': data['wind']['speed']
  }

Read historical **data**

In [19]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()  #remove row with missing values
  df = df.drop_duplicates()
  # print("Columns in historical data:", df.columns) # Add this line to display columns
  return df

**Prepare data for training**

In [20]:
def prepare_data(data):
  le = LabelEncoder() # create a encoder instance

  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define feature variable and target variable
  X = data[['MinTemp', 'MaxTemp','WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
  Y = data['RainTomorrow']  #target variable

  return X , Y, le

Train rain prediction **model**

In [21]:
#0.2 --> 20% testing, 80% training
def train_rain_model(X,Y):
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, Y_train)   #train the model

  Y_pred = model.predict(X_test)      #make prediction on test set

  print("Mean squared Error for Rain Model")

  print(mean_squared_error(Y_test, Y_pred))

  return model


**Prepare regression data**  --> it help to predict continuous values

In [22]:
def prepare_regression_data(data, feature):
  X, Y = [], []  #initilize list for feature and target values

  for i in range(len(data) -1):
    X.append(data[feature].iloc[i])

    Y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)   #2D array
  Y = np.array(Y)

  return X, Y


**Train regression model**

In [23]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100 , random_state=42)
  model.fit(x,y)
  return model

**Predict Future**

In [24]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])
  return predictions[1:]

** weather view**

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
def weather_view():
  city = input("enter city name : ")
  current_weather = get_current_weather(city)

  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model
  X, Y, le = prepare_data(historical_data)

  rain_model = train_rain_model(X, Y)

  #map wind direction to campus points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

  # Encode the compass direction, handling cases where the direction might not be in the training data
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1


  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['Wind_Gust_Speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  #rain prediction
  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for temperature and humdity
  X_temp, Y_temp = prepare_regression_data(historical_data, 'Temp')

  X_hum, Y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, Y_temp)

  hum_model = train_regression_model(X_hum, Y_hum)


  #predict future temperature and humidty
  future_temp = predict_future(temp_model, current_weather['temp_min'])

  future_humidity = predict_future(hum_model, current_weather['humidity'])

  #prepare time for future predictions
  timezone = pytz.timezone('Asia/kolkata')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #display results

  print(f"city: {city}, {current_weather['country']}")
  print(f"Wind Gusts : {current_weather['Wind_Gust_Speed']}")
  print(f"current temperature: {current_weather['current_temp']}")
  print(f"Feels like: {current_weather['feels_like']}")
  print(f"Minimum Temperature: {current_weather['temp_min']}°C")
  print(f"Maximum Temperature: {current_weather['temp_max']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'yes' if rain_prediction else 'No'}")

  print("\n Future Temperature Predictions : ")

  for time, temp in zip(future_times, future_temp):
    print(f"{time} : {round(temp, 1)}°C")

  print("\nFuture Humidity Prediction : ")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time} : {round(humidity, 1)}%")



**Running the Final **

In [27]:
weather_view()

enter city name : delhi
Mean squared Error for Rain Model
0.1506849315068493
city: delhi, IN
Wind Gusts : 3.6
current temperature: 31
Feels like: 38
Minimum Temperature: 31°C
Maximum Temperature: 31°C
Humidity: 74%
Weather Prediction: haze
Rain Prediction: yes

 Future Temperature Predictions : 
11:00 : 31.6°C
12:00 : 28.0°C
13:00 : 27.8°C
14:00 : 25.9°C
15:00 : 28.9°C

Future Humidity Prediction : 
11:00 : 44.8%
12:00 : 46.2%
13:00 : 48.2%
14:00 : 53.7%
15:00 : 58.1%
